In [1134]:
def evaluate_and_predict_final(X_train, X_test, y_train, y_test, test_final, submission_file="submission.csv"):
    """
    Evaluates models on train/test split.
    Uses best model to predict on separate test_final (no target).
    test_final must include Item_Identifier & Outlet_Identifier.
    Saves predictions to CSV.
    """
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.neural_network import MLPRegressor
    from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
    import pandas as pd

    models = {
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
        'GradientBoosting': GradientBoostingRegressor( random_state=42),
        'MLPRegressor': MLPRegressor(hidden_layer_sizes=(1024, 512, 256, 128), max_iter=500, random_state=42)
    }

    results = []
    model_store = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        rmse = mean_squared_error(y_test, preds, squared=False)

        results.append({
            'Model': name,
            'MAE': round(mean_absolute_error(y_test, preds), 2),
            'MAPE': round(mean_absolute_percentage_error(y_test, preds) * 100, 2),
            'RMSE': round(rmse, 2),
            'R2 Score': round(r2_score(y_test, preds), 4)
        })

        model_store[name] = model

    results_df = pd.DataFrame(results)
    best_model_name = results_df.sort_values(by='RMSE').iloc[0]['Model']
    best_model = model_store[best_model_name]

    print(f"✅ Best model: {best_model_name} (RMSE: {results_df.loc[results_df['Model'] == best_model_name, 'RMSE'].values[0]})")

    # Drop ID cols before prediction

    test_preds = best_model.predict(test_final)

    # Save submission
    submission = pd.DataFrame({
        'Item_Identifier': test['Item_Identifier'],
        'Outlet_Identifier': test['Outlet_Identifier'],
        'Item_Outlet_Sales': np.clip(test_preds, 0, None)
    })

    submission.to_csv(submission_file, index=False)
    print(f"📁 Submission saved to: {submission_file}")

    return results_df


In [1131]:
test_final=test[predictors]

In [1132]:
results = evaluate_and_predict_final(X_train, X_test, y_train, y_test, test_final=test_final)


✅ Best model: MLPRegressor (RMSE: 1067.28)
📁 Submission saved to: submission.csv


In [1133]:
results

,MAE,MAPE,Model,R2 Score,RMSE
0,870.56,103.86,LinearRegression,0.5052,1168.38
1,870.46,103.81,Ridge,0.5052,1168.35
2,870.34,103.62,Lasso,0.5054,1168.21
3,1007.43,70.94,DecisionTree,0.2433,1444.88
4,763.88,58.71,RandomForest,0.5625,1098.68
5,841.90,69.25,GradientBoosting,0.4789,1199.08
6,748.59,62.47,MLPRegressor,0.5871,1067.28


## 700

In [192]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

class TorchMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.ReLU(),
            nn.Linear(2048, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

def evaluate_and_predict_final(X_train, X_test, y_train, y_test, test_final, test_ids, submission_file="submission.csv"):
    """
    Trains PyTorch MLP on X_train/y_train, evaluates on X_test/y_test.
    Predicts on test_final and saves predictions to CSV with test_ids.
    """

    # Ensure numpy
    X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
    y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
    y_test = y_test.values if isinstance(y_test, pd.Series) else y_test
    test_final = test_final.values if isinstance(test_final, pd.DataFrame) else test_final

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

    model = TorchMLP(X_train.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("🔥 Training PyTorch MLP...")
    for epoch in range(1, 2000):
        model.train()
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_preds = model(X_test_tensor).cpu().numpy().flatten()
                val_true = y_test_tensor.cpu().numpy().flatten()
                rmse = mean_squared_error(val_true, val_preds, squared=False)
                r2 = r2_score(val_true, val_preds)
                mape = mean_absolute_percentage_error(val_true, val_preds) * 100

            print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | RMSE: {rmse:.2f} | R²: {r2:.4f} | MAPE: {mape:.2f}%")

    # Prediction on test_final
    model.eval()
    X_final_tensor = torch.tensor(test_final, dtype=torch.float32).to(device)
    with torch.no_grad():
        final_preds = model(X_final_tensor).cpu().numpy().flatten()
        final_preds = np.clip(final_preds, 0, None)

    # Save submission
    submission = pd.DataFrame({
        'Item_Identifier': test_ids['Item_Identifier'].values,
        'Outlet_Identifier': test_ids['Outlet_Identifier'].values,
        'Item_Outlet_Sales': final_preds
    })

    submission.to_csv(submission_file, index=False)
    print(f"✅ Submission saved to: {submission_file}")


In [193]:
evaluate_and_predict_final(
    X_train, X_test, y_train, y_test,
    test_final=test[predictors],
    test_ids=test[['Item_Identifier', 'Outlet_Identifier']]
)


🔥 Training PyTorch MLP...
Epoch 001 | Loss: 6284672.0000 | RMSE: 2530.58 | R²: -1.8935 | MAPE: 99.85%
Epoch 010 | Loss: 3340672.5000 | RMSE: 1512.85 | R²: -0.0341 | MAPE: 93.44%
Epoch 020 | Loss: 2068844.1250 | RMSE: 1456.12 | R²: 0.0420 | MAPE: 96.94%
Epoch 030 | Loss: 1646004.5000 | RMSE: 1233.25 | R²: 0.3128 | MAPE: 142.88%
Epoch 040 | Loss: 1529270.1250 | RMSE: 1236.87 | R²: 0.3088 | MAPE: 145.91%
Epoch 050 | Loss: 1542834.8750 | RMSE: 1233.56 | R²: 0.3124 | MAPE: 126.95%
Epoch 060 | Loss: 1514433.5000 | RMSE: 1228.96 | R²: 0.3176 | MAPE: 128.25%
Epoch 070 | Loss: 1503066.1250 | RMSE: 1220.29 | R²: 0.3272 | MAPE: 133.60%
Epoch 080 | Loss: 1494068.6250 | RMSE: 1216.64 | R²: 0.3312 | MAPE: 135.42%
Epoch 090 | Loss: 1482726.3750 | RMSE: 1213.24 | R²: 0.3349 | MAPE: 135.05%
Epoch 100 | Loss: 1471952.2500 | RMSE: 1209.94 | R²: 0.3385 | MAPE: 134.49%
Epoch 110 | Loss: 1460177.6250 | RMSE: 1206.36 | R²: 0.3424 | MAPE: 133.84%
Epoch 120 | Loss: 1445644.7500 | RMSE: 1201.89 | R²: 0.3473 | M

Epoch 1100 | Loss: 1134294.5000 | RMSE: 1075.46 | R²: 0.4774 | MAPE: 107.89%
Epoch 1110 | Loss: 1119472.2500 | RMSE: 1062.87 | R²: 0.4896 | MAPE: 92.45%
Epoch 1120 | Loss: 1101194.1250 | RMSE: 1055.02 | R²: 0.4971 | MAPE: 88.10%
Epoch 1130 | Loss: 1088221.6250 | RMSE: 1046.89 | R²: 0.5048 | MAPE: 86.34%
Epoch 1140 | Loss: 1079073.6250 | RMSE: 1040.87 | R²: 0.5105 | MAPE: 83.43%
Epoch 1150 | Loss: 1072185.2500 | RMSE: 1036.52 | R²: 0.5145 | MAPE: 79.59%
Epoch 1160 | Loss: 1063930.0000 | RMSE: 1031.70 | R²: 0.5191 | MAPE: 76.19%
Epoch 1170 | Loss: 1041651.0000 | RMSE: 1019.47 | R²: 0.5304 | MAPE: 69.80%
Epoch 1180 | Loss: 1018089.9375 | RMSE: 1006.80 | R²: 0.5420 | MAPE: 60.57%
Epoch 1190 | Loss: 985911.9375 | RMSE: 990.18 | R²: 0.5570 | MAPE: 58.30%
Epoch 1200 | Loss: 964674.1875 | RMSE: 981.69 | R²: 0.5645 | MAPE: 57.67%
Epoch 1210 | Loss: 953130.5625 | RMSE: 976.97 | R²: 0.5687 | MAPE: 57.95%
Epoch 1220 | Loss: 947768.2500 | RMSE: 975.27 | R²: 0.5702 | MAPE: 55.10%
Epoch 1230 | Loss: 

In [136]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

# ---------------------- MLP Definition ----------------------
import torch.nn as nn

class TorchMLP(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.3):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),

            nn.Linear(4096, 2048),
            nn.ReLU(),

            nn.Linear(2048, 1024),
            nn.ReLU(),

            nn.Linear(1024, 768),
            nn.ReLU(),

            nn.Linear(768, 512),
            nn.ReLU(),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),  # Dropout added here (deeper stage)

            nn.Linear(256, 128),
            nn.ReLU(),

    
            nn.Linear(128, 64),
            nn.Dropout(dropout_rate), 
            nn.ReLU(),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)



# ---------------------- Training + Prediction ----------------------
def evaluate_and_predict_final(X_train, X_test, y_train, y_test, test_final, test_ids, submission_file="submission.csv", model_path="best_model.pt"):
    """
    Trains PyTorch MLP on X_train/y_train, evaluates on X_test/y_test.
    Saves best model (lowest RMSE) to model_path.
    Predicts on test_final and saves submission to CSV with test_ids.
    """
    # Ensure numpy
    X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
    y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
    y_test = y_test.values if isinstance(y_test, pd.Series) else y_test
    test_final = test_final.values if isinstance(test_final, pd.DataFrame) else test_final

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

    model = TorchMLP(X_train.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_rmse = float('inf')

    print("🔥 Training PyTorch MLP...")
    for epoch in range(1, 2000):
        model.train()
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_preds = model(X_test_tensor).cpu().numpy().flatten()
                val_true = y_test_tensor.cpu().numpy().flatten()
                rmse = mean_squared_error(val_true, val_preds, squared=False)
                r2 = r2_score(val_true, val_preds)
                mape = mean_absolute_percentage_error(val_true, val_preds) * 100

                print(f"Epoch {epoch:04d} | Loss: {loss.item():.4f} | RMSE: {rmse:.2f} | R²: {r2:.4f} | MAPE: {mape:.2f}%")

                if rmse < best_rmse:
                    best_rmse = rmse
                    torch.save(model.state_dict(), model_path)
                    print(f"💾 Best model saved at epoch {epoch} (RMSE: {rmse:.2f})")

    # 🔁 Reload best model before final prediction
    model.load_state_dict(torch.load(model_path))
    model.eval()

    X_final_tensor = torch.tensor(test_final, dtype=torch.float32).to(device)
    with torch.no_grad():
        final_preds = model(X_final_tensor).cpu().numpy().flatten()
        final_preds = np.clip(final_preds, 0, None)

    # Save submission
    submission = pd.DataFrame({
        'Item_Identifier': test_ids['Item_Identifier'].values,
        'Outlet_Identifier': test_ids['Outlet_Identifier'].values,
        'Item_Outlet_Sales': final_preds
    })

    submission.to_csv(submission_file, index=False)
    print(f"✅ Submission saved to: {submission_file}")
    print(f"📦 Best model weights saved to: {model_path}")


In [132]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [133]:
gc.collect()
torch.cuda.empty_cache()


In [134]:
import torch
torch.cuda.empty_cache()

In [135]:
evaluate_and_predict_final(
    X_train, X_test, y_train, y_test,
    test_final=test[predictors],
    test_ids=test[['Item_Identifier', 'Outlet_Identifier']]
)


🔥 Training PyTorch MLP...
Epoch 0001 | Loss: 7842857.0000 | RMSE: 2673.48 | R²: -1.5906 | MAPE: 99.96%
💾 Best model saved at epoch 1 (RMSE: 2673.48)
Epoch 0010 | Loss: 4742855.0000 | RMSE: 2086.57 | R²: -0.5780 | MAPE: 270.21%
💾 Best model saved at epoch 10 (RMSE: 2086.57)
Epoch 0020 | Loss: 3285808.5000 | RMSE: 2036.82 | R²: -0.5037 | MAPE: 171.10%
💾 Best model saved at epoch 20 (RMSE: 2036.82)
Epoch 0030 | Loss: 3019185.2500 | RMSE: 1675.83 | R²: -0.0179 | MAPE: 152.78%
💾 Best model saved at epoch 30 (RMSE: 1675.83)
Epoch 0040 | Loss: 2758077.5000 | RMSE: 1607.87 | R²: 0.0630 | MAPE: 173.83%
💾 Best model saved at epoch 40 (RMSE: 1607.87)
Epoch 0050 | Loss: 2426232.2500 | RMSE: 1501.09 | R²: 0.1833 | MAPE: 137.39%
💾 Best model saved at epoch 50 (RMSE: 1501.09)
Epoch 0060 | Loss: 2203018.2500 | RMSE: 1491.75 | R²: 0.1934 | MAPE: 113.40%
💾 Best model saved at epoch 60 (RMSE: 1491.75)
Epoch 0070 | Loss: 2031223.5000 | RMSE: 1373.33 | R²: 0.3164 | MAPE: 106.59%
💾 Best model saved at epoch

Epoch 0950 | Loss: 1176650.7500 | RMSE: 1143.67 | R²: 0.5259 | MAPE: 47.26%
Epoch 0960 | Loss: 1181176.7500 | RMSE: 1258.92 | R²: 0.4256 | MAPE: 45.50%
Epoch 0970 | Loss: 1170875.3750 | RMSE: 1154.97 | R²: 0.5165 | MAPE: 46.74%
Epoch 0980 | Loss: 1164911.5000 | RMSE: 1197.52 | R²: 0.4802 | MAPE: 45.75%
Epoch 0990 | Loss: 1160203.2500 | RMSE: 1186.81 | R²: 0.4895 | MAPE: 45.81%
Epoch 1000 | Loss: 1164831.6250 | RMSE: 1175.28 | R²: 0.4994 | MAPE: 46.27%
Epoch 1010 | Loss: 1170745.0000 | RMSE: 1161.88 | R²: 0.5107 | MAPE: 47.06%
Epoch 1020 | Loss: 1169717.2500 | RMSE: 1234.82 | R²: 0.4473 | MAPE: 45.46%
Epoch 1030 | Loss: 1163270.5000 | RMSE: 1191.68 | R²: 0.4853 | MAPE: 45.78%
Epoch 1040 | Loss: 1164989.0000 | RMSE: 1210.82 | R²: 0.4686 | MAPE: 45.54%
Epoch 1050 | Loss: 1165050.1250 | RMSE: 1184.61 | R²: 0.4914 | MAPE: 45.89%
Epoch 1060 | Loss: 1170838.3750 | RMSE: 1150.75 | R²: 0.5200 | MAPE: 47.02%
Epoch 1070 | Loss: 1158922.5000 | RMSE: 1153.80 | R²: 0.5175 | MAPE: 46.52%
Epoch 1080 |

In [83]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score

import torch
import torch.nn as nn

class TorchMLP(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.3):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.BatchNorm1d(4096),     # ✅ Normalize after first big projection
            nn.ReLU(),

            nn.Linear(4096, 2048),
            nn.ReLU(),

            nn.Linear(2048, 1024),
            nn.ReLU(),

            nn.Linear(1024, 768),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(768, 512),
            nn.ReLU(),

            nn.Linear(512, 256),
            nn.ReLU(),

            nn.Linear(256, 128),
            nn.ReLU(),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)






# ---------------------- Training + Prediction ----------------------
def evaluate_and_predict_final(X_train, X_test, y_train, y_test, test_final, test_ids, submission_file="submission.csv", model_path="best_model.pt"):
    """
    Trains PyTorch MLP on X_train/y_train, evaluates on X_test/y_test.
    Saves best model (lowest RMSE) to model_path.
    Predicts on test_final and saves submission to CSV with test_ids.
    """
    # Ensure numpy
    X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
    y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
    y_test = y_test.values if isinstance(y_test, pd.Series) else y_test
    test_final = test_final.values if isinstance(test_final, pd.DataFrame) else test_final

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

    model = TorchMLP(X_train.shape[1]).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    best_rmse = float('inf')

    print("🔥 Training PyTorch MLP...")
    for epoch in range(1, 2000):
        model.train()
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = criterion(output, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                val_preds = model(X_test_tensor).cpu().numpy().flatten()
                val_true = y_test_tensor.cpu().numpy().flatten()
                rmse = mean_squared_error(val_true, val_preds, squared=False)
                r2 = r2_score(val_true, val_preds)
                mape = mean_absolute_percentage_error(val_true, val_preds) * 100

                print(f"Epoch {epoch:04d} | Loss: {loss.item():.4f} | RMSE: {rmse:.2f} | R²: {r2:.4f} | MAPE: {mape:.2f}%")

                if rmse < best_rmse:
                    best_rmse = rmse
                    torch.save(model.state_dict(), model_path)
                    print(f"💾 Best model saved at epoch {epoch} (RMSE: {rmse:.2f})")

    # 🔁 Reload best model before final prediction
    model.load_state_dict(torch.load(model_path))
    model.eval()

    X_final_tensor = torch.tensor(test_final, dtype=torch.float32).to(device)
    with torch.no_grad():
        final_preds = model(X_final_tensor).cpu().numpy().flatten()
        final_preds = np.clip(final_preds, 0, None)

    # Save submission
    submission = pd.DataFrame({
        'Item_Identifier': test_ids['Item_Identifier'].values,
        'Outlet_Identifier': test_ids['Outlet_Identifier'].values,
        'Item_Outlet_Sales': final_preds
    })

    submission.to_csv(submission_file, index=False)
    print(f"✅ Submission saved to: {submission_file}")
    print(f"📦 Best model weights saved to: {model_path}")


In [84]:
evaluate_and_predict_final(
    X_train, X_test, y_train, y_test,
    test_final=test[predictors],
    test_ids=test[['Item_Identifier', 'Outlet_Identifier']]
)

🔥 Training PyTorch MLP...
Epoch 0001 | Loss: 7843218.5000 | RMSE: 2673.67 | R²: -1.5910 | MAPE: 99.99%
💾 Best model saved at epoch 1 (RMSE: 2673.67)
Epoch 0010 | Loss: 6886391.0000 | RMSE: 2043.96 | R²: -0.5142 | MAPE: 90.02%
💾 Best model saved at epoch 10 (RMSE: 2043.96)
Epoch 0020 | Loss: 3607988.7500 | RMSE: 1620.59 | R²: 0.0481 | MAPE: 123.34%
💾 Best model saved at epoch 20 (RMSE: 1620.59)
Epoch 0030 | Loss: 3212147.7500 | RMSE: 1702.32 | R²: -0.0504 | MAPE: 108.81%
Epoch 0040 | Loss: 2760063.0000 | RMSE: 1675.18 | R²: -0.0171 | MAPE: 137.62%
Epoch 0050 | Loss: 2488994.0000 | RMSE: 1556.02 | R²: 0.1224 | MAPE: 142.87%
💾 Best model saved at epoch 50 (RMSE: 1556.02)
Epoch 0060 | Loss: 2260628.7500 | RMSE: 1473.58 | R²: 0.2130 | MAPE: 123.85%
💾 Best model saved at epoch 60 (RMSE: 1473.58)
Epoch 0070 | Loss: 2068158.0000 | RMSE: 1413.29 | R²: 0.2760 | MAPE: 113.70%
💾 Best model saved at epoch 70 (RMSE: 1413.29)
Epoch 0080 | Loss: 1877611.7500 | RMSE: 1330.54 | R²: 0.3583 | MAPE: 102.72

Epoch 0930 | Loss: 1163257.7500 | RMSE: 1038.51 | R²: 0.6091 | MAPE: 55.83%
Epoch 0940 | Loss: 1173487.2500 | RMSE: 1052.81 | R²: 0.5983 | MAPE: 62.21%
Epoch 0950 | Loss: 1166030.2500 | RMSE: 1045.57 | R²: 0.6038 | MAPE: 50.72%
Epoch 0960 | Loss: 1159492.8750 | RMSE: 1039.53 | R²: 0.6083 | MAPE: 56.48%
Epoch 0970 | Loss: 1156317.1250 | RMSE: 1040.10 | R²: 0.6079 | MAPE: 56.52%
Epoch 0980 | Loss: 1179167.7500 | RMSE: 1066.87 | R²: 0.5874 | MAPE: 65.82%
Epoch 0990 | Loss: 1168699.8750 | RMSE: 1044.20 | R²: 0.6048 | MAPE: 51.74%
Epoch 1000 | Loss: 1167456.6250 | RMSE: 1039.63 | R²: 0.6083 | MAPE: 55.64%
Epoch 1010 | Loss: 1176913.5000 | RMSE: 1063.89 | R²: 0.5898 | MAPE: 64.19%
Epoch 1020 | Loss: 1173449.2500 | RMSE: 1047.58 | R²: 0.6022 | MAPE: 50.94%
Epoch 1030 | Loss: 1162018.1250 | RMSE: 1041.04 | R²: 0.6072 | MAPE: 57.21%
Epoch 1040 | Loss: 1156514.2500 | RMSE: 1038.90 | R²: 0.6088 | MAPE: 55.38%
Epoch 1050 | Loss: 1155027.3750 | RMSE: 1042.71 | R²: 0.6059 | MAPE: 56.66%
Epoch 1060 |